In [1]:
from langchain.schema import HumanMessage, SystemMessage
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
import pandas as pd

In [25]:
#load envrionment variables
load_dotenv()
openai_key = os.getenv("OPENAI_KEY")

In [42]:
# Setting Up llm
llm_model = "gpt-4o" #Model 3.5turbo & gpt-40-mini doesnt perform well for some cases
llm = ChatOpenAI(model = llm_model, api_key=openai_key)

In [18]:
llm.invoke("Hello, there")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 10, 'total_tokens': 20, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-3e6b2c0d-02ef-41c1-b0ba-42956594457a-0', usage_metadata={'input_tokens': 10, 'output_tokens': 10, 'total_tokens': 20, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [33]:
#read csv file
# df = pd.read_csv("./data/salaries_2023.csv").fillna(value=0)

# read csv file from google sheets
# data gangguan Up3 pringsewu 2024
sheet_id = '1AtIrWJce3yTCYtAv9D7X56cbBQbA9JDxpXMLdmdSwvE'
df = pd.read_csv(f'https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv')
df.head()


,Penyulang,GI/GH/REC,ULP Koreksi,Tanggal,Kode Gangguan,Uraian Penyebab,Penyebab,Lepas (jam),Beban (Amp),Rele Kerja,Arus Gangguan (Amp),Masuk (jam),Beban (Amp)_1,Count > 3 Jam,Lama Padam (Menit),ENS,Count As Gangguan?,Bulan,Minggu,Hari,Jam,Tahun,UPPER(KODEGANGGUAN),KODE GG TRFRM,Unnamed: 24
0,KECAPI,RECLOSER TANJUNG JATI,PRINGSEWU,01/01/2023,Ps,NaN,TIDAK DITEMUKAN,1:03,80,51N,NaN,1:04,80,0,1,42,1.0,1.0,1.0,1.0,1,2023.0,PS,NaN,CERAH
1,SEMUT,RECLOSER SIDOLUHUR,KALIREJO,01/01/2023,Ps,NaN,TIDAK DITEMUKAN,1:08,70,51N-S,198,1:09,70,0,1,36,1.0,1.0,1.0,1.0,1,2023.0,PS,NaN,CERAH
2,SULING,RECLOSER PAGELARAN,TALANG PADANG,01/01/2023,Ps,NaN,TIDAK DITEMUKAN,2:40,63,51N-R,179,2:41,63,0,1,33,1.0,1.0,1.0,1.0,3,2023.0,PS,NaN,CERAH
3,MAWAR,RECLOSER NEW ULUBELU,TALANG PADANG,01/01/2023,Ps,NaN,TIDAK DITEMUKAN,4:19,25,OS,NaN,4:20,25,0,1,13,1.0,1.0,1.0,1.0,4,2023.0,PS,NaN,CERAH
4,PETRUK,RECLOSER BUMI ILIR,KALIREJO,01/01/2023,Ps,NaN,TIDAK DITEMUKAN,13:50,59,51N-R,128,13:51,58,0,1,31,1.0,1.0,1.0,1.0,14,2023.0,PS,NaN,CERAH


In [35]:
df['Penyulang'].value_counts()

Penyulang
BEO 2        499
POLENG 2     403
KECAPI       396
MAWAR        351
KACER        344
MURAI        311
SEMUT        291
GENDANG      287
MERCY        235
PIANO        235
KUMBANG      228
MOCCA        206
BELALANG     197
HARMONIKA    188
GITAR        174
COKLAT       172
LIBERIKA     171
LADA         168
ANGKLUNG     167
BIOLA        166
AMERICANO    165
GTA11        158
UKULELE 3    153
TROMPET      143
GTA12        138
BEO          136
ROBUSTA      125
PETRUK       118
CAMAR        105
CAMAR 2      105
UKULELE 2    104
CITROEN       99
GAGAK 2       98
CAPPUCINO     82
LATTE         70
MELATI        60
ELANG         58
SULING        57
CELLO         56
GAGAK         25
GTA13         22
GTA15         14
GTA14         12
UKULELE        8
KENARI         8
GTA16          8
ANGGREK        3
Name: count, dtype: int64

In [40]:
# Create Agent 
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent,create_csv_agent

agent = create_pandas_dataframe_agent(
    llm=llm,
    df = df_gs,
    verbose = True,
    allow_dangerous_code=True # Due to some warnings, we need to set this to True
)

# Adding prefic and suffix

CSV_PROMPT_PREFIX = """
First set the pandas display options to show all the columns,
get the column names, then answer the question.
"""

CSV_PROMPT_SUFFIX = """
- **ALWAYS** before giving the Final Answer, try another method.
Then reflect on the answers of the two methods you did and ask yourself
if it answers correctly the original question.
If you are not sure, try another method.
FORMAT 4 FIGURES OR MORE WITH COMMAS.
- If the methods tried do not give the same result,reflect and
try again until you have two methods that have the same result.
- If you still cannot arrive to a consistent result, say that
you are not sure of the answer.
- If you are sure of the correct answer, create a beautiful
and thorough response using Markdown.
- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,
ONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**.
- **ALWAYS**, as part of your "Final Answer", explain how you got
to the answer on a section that starts with: "\n\nExplanation:\n".
In the explanation, mention the column names that you used to get
to the final answer.
"""

In [47]:
# QUESTION = "What are the most TOP 10 count in 'Penyulang' columns?"
# QUESTION = "Hitung jumlah penyebab gangguan di kolom Penyebab untuk Penyulang Kecapi"
QUESTION = "Tampilkan ringkasan pada kolm Penyulang, Penyebab, dan Durasi"
res = agent.invoke(CSV_PROMPT_PREFIX + QUESTION + CSV_PROMPT_SUFFIX)



> Entering new AgentExecutor chain...

Thought: To complete the task, I'll first set the pandas display option to show all columns. Then, I'll retrieve the column names to better understand the dataframe structure. Next, I'll provide a summary for the specified columns: "Penyulang," "Penyebab," and "Lama Padam (Menit)." As instructed, I will use two different methods to ensure accuracy.

Action: python_repl_ast
Action Input: 
```python
import pandas as pd
pd.set_option('display.max_columns', None)
columns = df.columns
columns
```
Index(['Penyulang', 'GI/GH/REC', 'ULP Koreksi', 'Tanggal', 'Kode Gangguan',
       'Uraian Penyebab', 'Penyebab', 'Lepas (jam)', 'Beban (Amp)',
       'Rele Kerja', 'Arus Gangguan (Amp)', 'Masuk (jam)', 'Beban (Amp)_1',
       'Count > 3 Jam', 'Lama Padam (Menit)', 'ENS', 'Count As Gangguan?',
       'Bulan', 'Minggu', 'Hari', 'Jam', 'Tahun', 'UPPER(KODEGANGGUAN)',
       'KODE GG TRFRM', 'Unnamed: 24'],
      dtype='object')The dataframe contains several co